### Imports

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score

### Process the Dataset

In [ ]:
import pandas as pd

# Read the dataset CSV file into a DataFrame
df = pd.read_csv('dataset.csv')

# Read the testset CSV file into a DataFrame
test_data = pd.read_csv('testset.csv')

# Randomly shuffle the rows of the dataset DataFrame
df = df.sample(frac=1)

# Rename the columns to comply with the framework's requirements
df = df.rename(columns={'Sentence': 'text', 'label': 'labels'})

# Rename the columns to comply with the framework's requirements
test_data = test_data.rename(columns={'Sentence': 'text', 'Label': 'labels'})

In [ ]:
# Initialize a LabelEncoder object
le = LabelEncoder()

# Encode the 'labels' column in the DataFrame
df['label_encoded'] = le.fit_transform(df['labels'])

# Drop the original 'labels' column from the DataFrame
df = df.drop('labels', axis=1)

# Rename the columns in the DataFrame
df = df.rename(columns={'text': 'text', 'label_encoded': 'labels'})

In [ ]:
# Encode the 'labels' column in the test_data DataFrame
test_data['label_encoded'] = le.fit_transform(test_data['labels'])

# Drop the original 'labels' column from the test_data DataFrame
test_data = test_data.drop('labels', axis=1)

# Rename the columns in the test_data DataFrame
test_data = test_data.rename(columns={'text': 'text', 'label_encoded': 'labels'})

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Finetune the Model

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# Set the model arguments
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir=True, max_seq_length=512)

# Create a ClassificationModel using BERT
model_roberta = ClassificationModel("bert", "bert-base-multilingual-cased", args=model_args, num_labels=5)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

In [ ]:
# Train the model
model_roberta.train_model(df)

# Evaluate the model
result, model_outputs, wrong_predictions = model_roberta.eval_model(test_data)

  0%|          | 0/193 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 3:   0%|          | 0/25 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

## Evaluate the Model

In [ ]:
predictions, raw_outputs = model_roberta.predict(test_data['text'].tolist())
true_labels = test_data['labels'].tolist()

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
def f1_score(precision, recall):
    return 2 * ((precision * recall) / (precision + recall))

In [ ]:
precision = precision_score(true_labels, predictions, average = 'macro')
recall = recall_score(true_labels, predictions, average = 'macro')
accuracy = accuracy_score(true_labels, predictions)

print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
f1 = f1_score(precision, recall)
print("F1 score Aug:", f1)

Precision: 0.6999389499389499
Recall: 0.6478947368421053
Accuracy: 0.6464646464646465
F1 score Aug: 0.6729120457869975
